# Prep work over dashboard data

In [1]:
import requests
import json

In [2]:
# Load notebook with functions for RW api management

%run '/home/jovyan/work/notebooks-wri/layer-migration-v4/RW-API_management_functions.ipynb'

/home/jovyan/work/notebooks-wri/PreP
builtins==None
builtins==None
requests==2.26.0
json==2.0.9
requests_toolbelt.sessions==None
types==None
logging==0.5.1.2
getpass==None


In [27]:
# session for the RW api
session = rwApiSession()

You are login into RW api
Email: alicia.arenzana@vizzuality.com
Password: ········
Successfully logged into RW api


In [3]:
dashboardsPrep = requests.get('https://prepdata.org/api/dashboards').json()

In [4]:
len(dashboardsPrep)

14

In [28]:
def getPrepLayerList(session):
    '''
    Gets a list of assets from the selected env or from the constrained dataset list
    For our needs we will ignore other includer than layer.
    '''
    url = '/v1/layer'
    payload={
        'env':'production',
        'application':'prep',
        'status':'saved',
        'page[size]':100
    }
    response = []
    data = session.get(url, params=payload).json()
    
    response.extend(data['data'])
    
    # If pagination is needed we will access the data chunk by chunk
    if data['meta']['total-pages'] > 1:
        for n in range(2, data['meta']['total-pages'] + 1):
            payload['page[number]'] = n
            response.extend(session.get(url, params=payload).json()['data'])
    
    return response

In [79]:
prepLayers = getPrepLayerList(session)

In [81]:
problematicLayers = []

for layer in prepLayers:
    if layer.get('attributes').get('provider')  in ['loca', 'nexgddp']:
        problematicLayers.append(layer.get('id'))
        
len(problematicLayers)

160

In [82]:
def tryRequest(url: str):
    if 'nexgddp' in url or '/loca' in url:
        print(url)
    r = requests.get(url)
    if r.status_code not in [200, 202]:
        print(r.status_code)
        print(url)

In [68]:
# lets first update this 2 problematic layers:

session.get('/v1/layer/9054b065-769b-450f-bbab-e97877c28fdd').json()

{'data': {'id': '9054b065-769b-450f-bbab-e97877c28fdd',
  'type': 'layer',
  'attributes': {'name': '1971-2000 to 2051-2080 Increase in Number of Extreme Heat Days',
   'slug': '1971-2000-to-2051-2080-Difference-in-Extreme-Heat-Days-RCP-45-Days-Change_1',
   'dataset': '345a74e4-bc34-4077-b1e6-9096e5d44a9f',
   'description': 'The projected change in extreme heat days in 2051-2080, relative to 1971-2000, under a lower emissions scenario (RCP 4.5). Extreme heat days represents the annual average count of days with heat greater than the 99th percentile of the baseline.',
   'application': ['prep'],
   'iso': [],
   'provider': 'leaflet',
   'userId': '594a8b5b3b0f7f271d303321',
   'default': True,
   'protected': False,
   'published': True,
   'env': 'production',
   'layerConfig': {'type': 'tileLayer',
    'service': 'leaflet',
    'url': 'https://api.resourcewatch.org/v1/layer/a341796d-ac66-443c-9eb9-27918b315ff7/tile/nexgddp/{z}/{x}/{y}?year=1971-01-01T00:00:00.000Z&compareYear=2051-

In [74]:
newLayerConfig = {
    'layerConfig': {
    'type': 'tileLayer',
    'service': 'leaflet',
    'url': 'https://api.resourcewatch.org/v1/layer/8e896bf3-b445-4efc-a23d-5552637ad263/tile/gee/{z}/{x}/{y}',
    'body': {'format': 'image/png', 'minZoom': 2, 'maxZoom': 13},
    'id': 'e8ba6f1d-e983-45cd-aee9-805284719cd2'
    }
}
json.dumps(newLayerConfig)
session.patch('/v1/dataset/345a74e4-bc34-4077-b1e6-9096e5d44a9f/layer/9054b065-769b-450f-bbab-e97877c28fdd', json = newLayerConfig).json()

'{"layerConfig": {"type": "tileLayer", "service": "leaflet", "url": "https://api.resourcewatch.org/v1/layer/8e896bf3-b445-4efc-a23d-5552637ad263/tile/gee/{z}/{x}/{y}", "body": {"format": "image/png", "minZoom": 2, "maxZoom": 13}, "id": "e8ba6f1d-e983-45cd-aee9-805284719cd2"}}'

In [75]:
session.get('/v1/layer/831b208e-cd43-42c2-a8bc-a6e2316afdde').json()

{'data': {'id': '831b208e-cd43-42c2-a8bc-a6e2316afdde',
  'type': 'layer',
  'attributes': {'name': '1971-2000 to 2021-2050 Increase in Number of Extreme Heat Days',
   'slug': '1971-2000-to-2021-2050-Difference-in-Extreme-Heat-Days-RCP-45-Days-Change',
   'dataset': 'd5358bba-a07c-47c9-ba7a-7b168a10cc75',
   'description': 'The projected change in extreme heat days in 2021-2050, relative to 1971-2000, under a lower emissions scenario (RCP 4.5). Extreme heat days represents the annual average count of days with heat greater than the 99th percentile of the baseline.',
   'application': ['prep'],
   'iso': [],
   'provider': 'leaflet',
   'userId': '594a8b5b3b0f7f271d303321',
   'default': True,
   'protected': False,
   'published': True,
   'env': 'production',
   'layerConfig': {'type': 'tileLayer',
    'service': 'leaflet',
    'url': 'https://api.resourcewatch.org/v1/layer/a341796d-ac66-443c-9eb9-27918b315ff7/tile/nexgddp/{z}/{x}/{y}?year=1971-01-01T00:00:00.000Z&compareYear=2021-01

In [77]:
newLayerConfig = {
    'layerConfig': {
    'type': 'tileLayer',
    'service': 'leaflet',
    'url': 'https://api.resourcewatch.org/v1/layer/a51fabc9-d31b-49c9-b417-0d5a3adbbc39/tile/gee/{z}/{x}/{y}',
    'body': {'format': 'image/png', 'minZoom': 2, 'maxZoom': 13},
    'id': 'e8ba6f1d-e983-45cd-aee9-805284719cd2'
    }
}
json.dumps(newLayerConfig)
session.patch('/v1/dataset/d5358bba-a07c-47c9-ba7a-7b168a10cc75/layer/831b208e-cd43-42c2-a8bc-a6e2316afdde', json = newLayerConfig).json()

{'data': {'id': '831b208e-cd43-42c2-a8bc-a6e2316afdde',
  'type': 'layer',
  'attributes': {'name': '1971-2000 to 2021-2050 Increase in Number of Extreme Heat Days',
   'slug': '1971-2000-to-2021-2050-Difference-in-Extreme-Heat-Days-RCP-45-Days-Change',
   'dataset': 'd5358bba-a07c-47c9-ba7a-7b168a10cc75',
   'description': 'The projected change in extreme heat days in 2021-2050, relative to 1971-2000, under a lower emissions scenario (RCP 4.5). Extreme heat days represents the annual average count of days with heat greater than the 99th percentile of the baseline.',
   'application': ['prep'],
   'iso': [],
   'provider': 'leaflet',
   'userId': '594a8b5b3b0f7f271d303321',
   'default': True,
   'protected': False,
   'published': True,
   'env': 'production',
   'layerConfig': {'type': 'tileLayer',
    'service': 'leaflet',
    'url': 'https://api.resourcewatch.org/v1/layer/a51fabc9-d31b-49c9-b417-0d5a3adbbc39/tile/gee/{z}/{x}/{y}',
    'body': {'format': 'image/png', 'minZoom': 2, '

In [83]:
def tryWidget(url: str):
    r = requests.get(url)
    if r.status_code not in [200, 202]:
        print(r.status_code)
        print(url)
        print(r.text)
    
    if r.json().get('data',{}).get('attributes',{}).get('widgetConfig',{}).get('type') == 'map':
        layerId = r.json().get('data',{}).get('attributes',{}).get('widgetConfig',{}).get('layer_id')
        if layerId in problematicLayers:
            print(f'https://api.resourcewatch.org/v1/layer/{layerId}')
        # s = requests.get(f'https://api.resourcewatch.org/v1/layer/{layerId}')
    
    

In [84]:
def recursiveGridCheck(asset: list):
    for gridContent in asset:
        if gridContent is not None:
            if type(gridContent) is list:
                recursiveGridCheck(gridContent)
            elif gridContent.get('type') == 'widget':
                tryWidget(f'https://api.resourcewatch.org/v1/widget/{gridContent.get("content").get("widgetId")}')
            elif gridContent.get('type') == 'embed':
                 tryRequest(gridContent.get('content').get('src'))
                

In [86]:
for dashboard in dashboardsPrep:
    print(dashboard['id'])
    contentTypes = {}
    for content in json.loads(dashboard.get('content')):
        if content.get('type') in contentTypes.keys():
            contentTypes[content.get('type')] += 1
        else:
            contentTypes[content.get('type')] = 1
        
        if content.get('type') == 'grid':
            recursiveGridCheck(content.get('content'))
        elif content.get('type') == 'embed':
            tryRequest(content.get('content').get('src'))
    
    print(contentTypes)
    print('______________________________________________________________________________')

229
https://www.prepdata.org/embed/nexgddp/xpr_rcp45_decadal?zoom=7&lat=9.973659811686428&lng=-84.0075682103634&mapMode=side-by-side&graphMode=timeseries&scenario=rcp85&tempResolution=30_y&range1=1971-01-01T00%3A00%3A00.000Z&range2=2051-01-01T00%3A00%3A00.000Z&basemap=light&labels=light&boundaries=true&render=map
{'text': 44, 'grid': 9, 'embed': 3, 'image': 1}
______________________________________________________________________________
263
{'text': 43, 'grid': 9, 'embed': 4}
______________________________________________________________________________
50
https://prepdata.org/embed/nexgddp/xpr_rcp45_decadal?zoom=7&lat=10.022948394510154&lng=-84.803466796875&markerLat=10.066220126753016&markerLng=-84.02893066406251&mapMode=difference&graphMode=timeseries&scenario=rcp85&tempResolution=30_y&range1=1971-01-01T00%3A00%3A00.000Z&range2=2021-01-01T00%3A00%3A00.000Z&labels=light&boundaries=true&render=map
{'text': 16, 'embed': 3, 'grid': 3, 'widget': 1}
______________________________________

In [30]:
len(prepLayers)

1750

In [31]:
providers = {}
for layer in prepLayers:
    if layer.get('attributes').get('provider') in providers.keys():
        providers[layer.get('attributes',{}).get('provider')] += 1
    else:
        providers[layer.get('attributes',{}).get('provider')] = 1
providers

{'arcgis': 102,
 'gee': 859,
 'cartodb': 539,
 'loca': 80,
 'nexgddp': 80,
 'featureservice': 5,
 'leaflet': 22,
 'wms': 63}

In [ ]:
session.get('/v1/widget/acfd8195-9729-4c88-afbb-731deffd1e55').json()

In [ ]:
session.get('/v1/layer/9054b065-769b-450f-bbab-e97877c28fdd').json()